# `dev_utils`
> Classes and functions used in development phase.

In [ ]:
#|default_exp dev_utils

In [ ]:
#| hide
from fastcore.test import test_fail
from nbdev import show_doc, nbdev_export
from pprint import pprint

In [ ]:
#| export
from __future__ import annotations
from pathlib import Path
from typing import Any
import inspect
import re
import sys
import functools

# Tracing operations

Classes and decorators to work with `sys.settrace(tracefunc)`

#### Technical note:

`tracefunc`(the trace function) should have three arguments: `frame`, `event`, and `arg`:

- `frame` is the current stack frame. 
- `event` is a string: `'call'`, `'line'`, `'return'`, `'exception'` or `'opcode'`.
- `arg` depends on the event type.

`frame` has many attributes, including those below which are used in the tracing classes below:

|Type | Attribute | Description |
|:----|:----------|:------------|
|frame|**`f_back`** | next outer frame object (this frame’s caller)|
|     |**`f_code`**| code object being executed in this frame
|     |**`f_lineno`** |  current line number in Python source code
|code |**`co_code`**| string of raw compiled bytecode
|     |**`co_filename`**| name of file in which this code object was created
|     |`co_name`|name with which this code object was defined
|     |`co_names`|tuple of names other than arguments and function locals
|     |`co_stacksize`|virtual machine stack space required
|     |`co_varnames`|tuple of names of arguments and local variables

See full documentation in [`sys`](https://docs.python.org/3/library/sys.html#sys.settrace) and [`inspect`](https://docs.python.org/3/library/inspect.html#types-and-membersl) built-in modules.

Experiments with frame and its attributes

In [ ]:
def fn(a):
    return a + 1

b = fn(1)

frames = sys._current_frames()

In [ ]:
framesids = list(frames.keys())

for id in framesids:
    fr = frames[id]
    print(f"{fr.f_lineno:5d}   {fr.f_code.co_name:20s}   {fr.f_code.co_filename:50s}")


   37   run                    /home/vtec/miniconda3/envs/ecutils/lib/python3.10/site-packages/ipykernel/parentpoller.py
  330   wait                   /home/vtec/miniconda3/envs/ecutils/lib/python3.10/threading.py
  482   select                 /home/vtec/miniconda3/envs/ecutils/lib/python3.10/selectors.py
  327   _watch_pipe_fd         /home/vtec/miniconda3/envs/ecutils/lib/python3.10/site-packages/ipykernel/iostream.py
  327   _watch_pipe_fd         /home/vtec/miniconda3/envs/ecutils/lib/python3.10/site-packages/ipykernel/iostream.py
  103   run                    /home/vtec/miniconda3/envs/ecutils/lib/python3.10/site-packages/ipykernel/heartbeat.py
  482   select                 /home/vtec/miniconda3/envs/ecutils/lib/python3.10/selectors.py
    6   <cell line: 6>         /tmp/ipykernel_7075/1808055364.py                 


In [ ]:
frame = frames[framesids[-1]]

print(frame.f_code.co_names)
print(frame.f_code.co_stacksize)
print(frame.f_code.co_varnames)

print(inspect.getsource(frame))

('sys', '_current_frames', 'frames')
2
()
def fn(a):
    return a + 1



## Tracing classes

In [ ]:
#| export
class StackTrace():
    """Callable class acting as `tracefunc` to capture and print information on all stack frame being run"""
    def __init__(self, 
                 with_call:bool=True,       # when True, `call` events are traced
                 with_return:bool=True,     # when True, `return` events are traced
                 with_exception:bool=True,  # when True, `exceptions` events are traced
                 max_depth:int=-1           # maximum depth of the trace, default is full depth
                ):
        self._frame_dict = {}
        self._options = set()
        self._max_depth = max_depth
        if with_call: self._options.add('call')
        if with_return: self._options.add('return')
        if with_exception: self._options.add('exception')

    def __call__(self, 
                 frame: inspect.FrameInfo,       # `frame` argument in tracefunc
                 event:str,   # `event` argument in tracefunc
                 arg:Any,     # `arg` argument in tracefunc
                ):
        """`tracefunc`used in `sys.settrace(tracefunc)`"""
        ret = []
        co_name = frame.f_code.co_name
        co_filename = frame.f_code.co_filename
        co_lineno = frame.f_lineno
        if event == 'call':
            back_frame = frame.f_back
            if back_frame in self._frame_dict:
                self._frame_dict[frame] = self._frame_dict[back_frame] + 1
            else:
                self._frame_dict[frame] = 0

        depth = self._frame_dict[frame]

        if event in self._options and (self._max_depth < 0 or depth <= self._max_depth):
            ret.append(co_name)
            ret.append(f'[{event}]')
            if event == 'return':
                ret.append(arg)
            elif event == 'exception':
                ret.append(repr(arg[0]))
            ret.append(f'in {co_filename} line:{co_lineno}')
        if ret:
            self.print_stack_info(co_filename, ret, depth)
        return self

    def print_stack_info(self, 
                         co_filename:str|Path, # code file name
                         ret:bool, # 
                         depth:int, # depth
                        ):
        """This methods can be overloaded to customize what is printed out"""
        text = '\t'.join([str(i) for i in ret])
        print(f"{'  ' * depth}{text}")

In [ ]:
show_doc(StackTrace.__call__)

---

[source](https://github.com/vtecftwy/ecutils/blob/master/ecutilities/dev_utils.py#L31){target="_blank" style="float:right; font-size:smaller"}

### StackTrace.__call__

>      StackTrace.__call__ (frame:inspect.FrameInfo, event:str, arg:Any)

`tracefunc`used in `sys.settrace(tracefunc)`

|    | **Type** | **Details** |
| -- | -------- | ----------- |
| frame | inspect.FrameInfo | `frame` argument in tracefunc |
| event | str | `event` argument in tracefunc |
| arg | Any | `arg` argument in tracefunc |

In [ ]:
show_doc(StackTrace.print_stack_info)

---

[source](https://github.com/vtecftwy/ecutils/blob/master/ecutilities/dev_utils.py#L62){target="_blank" style="float:right; font-size:smaller"}

### StackTrace.print_stack_info

>      StackTrace.print_stack_info (co_filename:str|pathlib.Path, ret:bool,
>                                   depth:int)

This methods can be overloaded to customize what is printed out

|    | **Type** | **Details** |
| -- | -------- | ----------- |
| co_filename | str \| Path | code file name |
| ret | bool |  |
| depth | int | depth |

In [ ]:
#| export
class StackTraceJupyter(StackTrace):
    """Print stack frame information in Jupyter notebook context (filters out jupyter overhead)"""

    def print_stack_info(self, 
                         co_filename, 
                         ret, 
                         depth
                        ):
        """Overload the base class to filter out those calls to Jupyter overhead functions"""

        EXCL_LIBS = ['encodings.*', 'ntpath.*', 'threading.*', 'weakref.*']
        EXCL_SITE_PACKAGES = ['colorama', 'ipykernel', 'zmq']

        PATH_TO_LIBS_RE = r'^[a-zA-Z]:\\([^<>:\"/\\|?\*]*)\\envs\\([^<>:\"/\\|?\*]*)\\lib'
        LIBS = f"{'|'.join(EXCL_LIBS)}"
        SITE_PACKAGES = f"{'|'.join(EXCL_SITE_PACKAGES)}"
        MODULE_FILTERS_RE = rf"{PATH_TO_LIBS_RE}\\(({LIBS})|(site-packages\\({SITE_PACKAGES}))\\.*)"

        pat = re.compile(MODULE_FILTERS_RE)
        match = pat.match(co_filename)
        
        if match is None:
            """Only print stack frame info for those objects where there is no match"""
            text = '\t'.join([str(i) for i in ret])
            print(f"{'  ' * depth}{text}")

## Tracing decorators

In [ ]:
#| export
def stack_trace(**kw):
    """`stack_trace` decorator function"""
    def entangle(func):
        @functools.wraps(func)
        def wrapper(*args, **kwargs):
            # st = StackTrace(**kw)
            st = StackTrace(**kw)
            sys.settrace(st)
            try:
                return func(*args, **kwargs)
            finally:
                sys.settrace(None)
        return wrapper
    return entangle

In [ ]:
show_doc(stack_trace)

---

[source](https://github.com/vtecftwy/ecutils/blob/master/ecutilities/dev_utils.py#L99){target="_blank" style="float:right; font-size:smaller"}

### stack_trace

>      stack_trace (**kw)

`stack_trace` decorator function

In [ ]:
#| export
def stack_trace_jupyter(**kw):
    """`stack_trace_jupyter` decorator function"""
    def entangle(func):
        @functools.wraps(func)
        def wrapper(*args, **kwargs):
            # st = StackTrace(**kw)
            st = StackTraceJupyter(**kw)
            sys.settrace(st)
            try:
                return func(*args, **kwargs)
            finally:
                sys.settrace(None)
        return wrapper
    return entangle

#### Usage:

Several functions, some of them nested and some of them with errors.

In [ ]:
def empty_func():
        pass

def call_empty_and_return_zero():
    empty_func()
    return 0

def divide_by_zero_error():
    1/0

def decrement_recursion(i):
    if i == 0:
        return
    decrement_recursion(i-1)

Using the `@stack_trace` or `@stack_trace_jupyter` decorator allows a detailled trace, function by function and identify where it fails.

In [ ]:
@stack_trace(with_return=True, with_exception=True, max_depth=10)
def function_to_trace():
    call_empty_and_return_zero()
    decrement_recursion(5)
    divide_by_zero_error()

In [ ]:
test_fail(
    function_to_trace,
    msg='Should raise a div by 0 exception',
    contains='division by zero'
)

function_to_trace	[call]	in /tmp/ipykernel_7075/661382525.py line:1
  call_empty_and_return_zero	[call]	in /tmp/ipykernel_7075/2653264264.py line:4
    empty_func	[call]	in /tmp/ipykernel_7075/2653264264.py line:1
    empty_func	[return]	None	in /tmp/ipykernel_7075/2653264264.py line:2
  call_empty_and_return_zero	[return]	0	in /tmp/ipykernel_7075/2653264264.py line:6
  decrement_recursion	[call]	in /tmp/ipykernel_7075/2653264264.py line:11
    decrement_recursion	[call]	in /tmp/ipykernel_7075/2653264264.py line:11
      decrement_recursion	[call]	in /tmp/ipykernel_7075/2653264264.py line:11
        decrement_recursion	[call]	in /tmp/ipykernel_7075/2653264264.py line:11
          decrement_recursion	[call]	in /tmp/ipykernel_7075/2653264264.py line:11
            decrement_recursion	[call]	in /tmp/ipykernel_7075/2653264264.py line:11
            decrement_recursion	[return]	None	in /tmp/ipykernel_7075/2653264264.py line:13
          decrement_recursion	[return]	None	in /tmp/ipykernel_70

In [ ]:
@stack_trace_jupyter(with_return=True, with_exception=True, max_depth=15)
def function_to_trace_jupyter():
    call_empty_and_return_zero()
    decrement_recursion(5)
    divide_by_zero_error()

In [ ]:
test_fail(
    function_to_trace_jupyter,
    msg='Should raise a div by 0 exception',
    contains='division by zero'
)

function_to_trace_jupyter	[call]	in /tmp/ipykernel_7075/2571630784.py line:1
  call_empty_and_return_zero	[call]	in /tmp/ipykernel_7075/2653264264.py line:4
    empty_func	[call]	in /tmp/ipykernel_7075/2653264264.py line:1
    empty_func	[return]	None	in /tmp/ipykernel_7075/2653264264.py line:2
  call_empty_and_return_zero	[return]	0	in /tmp/ipykernel_7075/2653264264.py line:6
  decrement_recursion	[call]	in /tmp/ipykernel_7075/2653264264.py line:11
    decrement_recursion	[call]	in /tmp/ipykernel_7075/2653264264.py line:11
      decrement_recursion	[call]	in /tmp/ipykernel_7075/2653264264.py line:11
        decrement_recursion	[call]	in /tmp/ipykernel_7075/2653264264.py line:11
          decrement_recursion	[call]	in /tmp/ipykernel_7075/2653264264.py line:11
            decrement_recursion	[call]	in /tmp/ipykernel_7075/2653264264.py line:11
            decrement_recursion	[return]	None	in /tmp/ipykernel_7075/2653264264.py line:13
          decrement_recursion	[return]	None	in /tmp/ipy

In [ ]:
#| hide
nbdev_export()